In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from brian2 import *
from brian2tools import *
import numpy as np
import pandas as pd


# Customize matplotlib
matplotlib.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
        'font.size': 22,
        'ytick.color' : 'black',
        "xtick.color" : 'black',
        'axes.labelcolor' : 'black',
        'axes.edgecolor' : 'black'
	}
)

In [7]:
n_points = 100
n_samples = 1
vec_nu = []
for nu_value in np.linspace(0.1, 100, n_points):
	for n in range(n_samples):
		vec_nu.append(nu_value)

vec_nu = np.array(vec_nu)

EL = -60*mV
EE = 0*mV
EI = -80*mV
alpha = 1
tauN = 100*ms
tauI = 10*ms
tauL = 20*ms

eqs = '''
dv/dt = ((EL - v) + gA*(EE - v) + gI*(EI - v))/tauL : volt
dgA/dt = -gA/tauA : 1
dgI/dt = -gI/tauI : 1
tauA : second
'''

tauA_list = [5, 10, 20, 50, 100]
mean_V_ta = {'nu':vec_nu, 5:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples), 20:np.zeros(n_points*n_samples), 50:np.zeros(n_points*n_samples), 100:np.zeros(n_points*n_samples)}
var_V_ta = {'nu':vec_nu, 5:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples), 20:np.zeros(n_points*n_samples), 50:np.zeros(n_points*n_samples), 100:np.zeros(n_points*n_samples)}

for tauA_value in tauA_list:
	print(tauA_value, end='\t')
	for i, nu_value in enumerate(vec_nu):
		start_scope()
		print(i/size(vec_nu), end=' ')
		G = NeuronGroup(1, eqs, method='rk4')
		GPE = PoissonGroup(400, rates=nu_value*Hz)
		GPI = PoissonGroup(100, rates=nu_value*Hz)

		SE = Synapses(GPE, G, 'w : 1', on_pre='gA += w')
		SI = Synapses(GPI, G, 'w : 1', on_pre='gI += w')
		SE.connect()
		SI.connect()

		G.v = EL
		SE.w = 0.1
		SI.w = 0.4

		G.tauA = tauA_value*ms

		run(5000*ms)

		M = StateMonitor(G, 'v', record=True)

		run(20000*ms)
		mean_V_ta[tauA_value][i] = np.mean(M.v/mV)
		var_V_ta[tauA_value][i] = np.var(M.v/mV)

	print()

5	0.0 0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1 0.11 0.12 0.13 0.14 0.15 0.16 0.17 0.18 0.19 0.2 0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3 0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4 0.41 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5 0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6 0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8 0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89 0.9 0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98 0.99 
10	0.0 0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1 0.11 0.12 0.13 0.14 0.15 0.16 0.17 0.18 0.19 0.2 0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3 0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4 0.41 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5 0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6 0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8 0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89 0.9 0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98 0.99 
20	0.0 0.01 0

In [25]:
# Convert to pandas DataFrame
df_mean_V_ta = pd.DataFrame.from_dict(mean_V_ta)
df_var_V_ta = pd.DataFrame.from_dict(var_V_ta)


# Save Dataframe do csv file
df_mean_V_ta.to_csv('mean_V_tauFixed.dat')
df_var_V_ta.to_csv('var_V_tauFixed.dat')


In [21]:
mean_V_ta['tauA'].size

2500